In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
print(os.listdir('./input/'))

['EDA(1)_test1_activity.csv', 'EDA(1)_test1_combat.csv', 'EDA(1)_test1_payment.csv', 'EDA(1)_test1_pledge.csv', 'EDA(1)_test1_source_trade.csv', 'EDA(1)_test1_target_trade.csv', 'EDA(1)_test2_activity.csv', 'EDA(1)_test2_combat.csv', 'EDA(1)_test2_payment.csv', 'EDA(1)_test2_pledge.csv', 'EDA(1)_train_activity.csv', 'EDA(1)_train_combat.csv', 'EDA(1)_train_payment.csv', 'EDA(1)_train_pledge.csv', 'EDA(1)_train_source_trade.csv', 'EDA(1)_train_target_trade.csv', 'EDA(2)_test1_combat.csv', 'EDA(2)_test2_combat.csv', 'EDA(2)_train_combat.csv', 'sort_train_label.csv']


In [4]:
tr_act = pd.read_csv('./input/EDA(1)_train_activity.csv')
tr_combat = pd.read_csv('./input/EDA(2)_train_combat.csv')
tr_pledge = pd.read_csv('./input/EDA(1)_train_pledge.csv')
tr_label = pd.read_csv('./Data/train_label.csv')

In [5]:
print(len(tr_act.acc_id.unique()))
print(len(tr_combat.acc_id.unique()))
print(len(tr_pledge.acc_id.unique()))

40000
40000
33854


In [6]:
train = tr_label.merge(tr_act, on='acc_id', how='outer')
train = train.merge(tr_combat, on='acc_id', how='outer')

In [7]:
t1_act = pd.read_csv('./input/EDA(1)_test1_activity.csv')
t1_combat = pd.read_csv('./input/EDA(2)_test1_combat.csv')
t1_pledge = pd.read_csv('./input/EDA(1)_test1_pledge.csv')

In [8]:
t2_act = pd.read_csv('./input/EDA(1)_test2_activity.csv')
t2_combat = pd.read_csv('./input/EDA(2)_test2_combat.csv')

In [9]:
test1 = t1_act.merge(t1_combat, on='acc_id', how='outer')

In [10]:
test1 = test1.drop_duplicates()

In [11]:
test2 = t2_act.merge(t2_combat, on='acc_id', how='outer')

In [12]:
test2 = test2.drop_duplicates()

In [13]:
train = train.drop('acc_id', axis = 1)
test01 = test1.drop('acc_id', axis = 1)
test02 = test2.drop('acc_id', axis = 1)

In [14]:
train = train.drop(['boss_monster', 'exp_recovery', 'enchant_count', 'random_defender_cnt', 'same_pledge_cnt'], axis = 1)
test01 = test01.drop(['boss_monster', 'exp_recovery', 'enchant_count', 'random_defender_cnt', 'same_pledge_cnt'], axis = 1)
test02 = test02.drop(['boss_monster', 'exp_recovery', 'enchant_count', 'random_defender_cnt', 'same_pledge_cnt'], axis = 1)

In [15]:
from sklearn.model_selection import train_test_split
X = train.drop(['survival_time', 'amount_spent'], axis = 1)
y = train['survival_time']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=50)


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

### 1. KNeighborsClassifier

In [18]:
KN = KNeighborsClassifier(3)
KN.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [19]:
KN_pred = KN.predict(X_test)


In [20]:
print('KN_survival_time : {}'.format(accuracy_score(KN_pred, y_test)))


KN_survival_time : 0.8159529887770338


### 2. SVC

In [21]:
# svc = SVC(gamma='auto')
# svc.fit(X_train, y_train)

In [22]:
# svc_pred = svc.predict(X_test)

In [23]:
# print('SVC_survival_time : {}'.format(accuracy_score(svc_pred, y_test)))

### ※ LinearSVC
  * SVC 모델 적용시 시간이 너무 오래 걸리는 불편함이 발생
  * scikit-learn document에서 구한 정보를 바탕으로 대규모 데이터 세트일 경우 사용하는 LinearSVC 활용

In [24]:
# from sklearn.svm import LinearSVC

In [25]:
# LSVC = LinearSVC(random_state=0)
# LSVC.fit(X_train, y_train)

In [26]:
# LSVC_pred = LSVC.predict(X_test)

In [27]:
# print()

### 3. GaussianProcessClassifier

In [28]:
# kernel = 1.0*RBF(1.0)
# GPC = GaussianProcessClassifier(kernel = kernel, random_state = 0)
# GPC.fit(X_train, y_train)

### 4. DecisionTreeClassifier

In [33]:
DTC = DecisionTreeClassifier(random_state=0, max_depth = 20, presort=True)
DTC.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=True,
                       random_state=0, splitter='best')

In [34]:
DTC_pred = DTC.predict(X_test)

In [35]:
print('DTC_survival_time : {}'.format(accuracy_score(DTC_pred, y_test)))

DTC_survival_time : 0.7713019604065314


### 5. RandomForestClassifier

In [36]:
RFC = RandomForestClassifier(random_state = 0, n_estimators=100, max_depth=2)
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [37]:
RFC_pred = RFC.predict(X_test)

In [38]:
print('RFC_survival_time : {}'.format(accuracy_score(RFC_pred, y_test)))

RFC_survival_time : 0.5604739656085931


### 6. AdaBoostClassifier

In [39]:
ABC = AdaBoostClassifier(n_estimators=100, random_state = 0)
ABC.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [40]:
ABC_pred = ABC.predict(X_test)

In [41]:
print('ABC_survival_time : {}'.format(accuracy_score(ABC_pred, y_test)))

ABC_survival_time : 0.5506478493328838


### 7. QuadraticDiscriminantAnalysis

In [42]:
QDA = QuadraticDiscriminantAnalysis()
QDA.fit(X_train, y_train)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)

In [43]:
QDA_pred = QDA.predict(X_test)

In [44]:
print('QDA : {}'.format(accuracy_score(QDA_pred, y_test)))

QDA : 0.10803911179615626


### 점수가 제일 높은 DecisionTreeClassifier 활용 예측

In [45]:
t1_DTC_pred = DTC.predict(test01)

In [46]:
t1_survival_submission = test1.drop(['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'boss_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'class_0', 'class_1',
       'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7'], axis = 1)

In [47]:
t1_survival_submission['survival_time'] = t1_DTC_pred

In [48]:
t1_survival_submission.to_csv('./test1_predict_survival_time.csv', encoding = 'UTF-8', header=True, index = False)

In [49]:
t2_DTC_pred = DTC.predict(test02)

In [50]:
t2_survival_submission = test2.drop(['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp',
       'boss_monster', 'death', 'revive', 'exp_recovery', 'fishing',
       'private_shop', 'game_money_change', 'enchant_count', 'pledge_cnt',
       'random_attacker_cnt', 'random_defender_cnt', 'temp_cnt',
       'same_pledge_cnt', 'etc_cnt', 'num_opponent', 'class_0', 'class_1',
       'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7'], axis = 1)

In [51]:
t2_survival_submission['survival_time'] = t2_DTC_pred

In [52]:
t2_survival_submission.to_csv('./test2_predict_survival_time.csv', encoding = 'UTF-8', header=True, index = False)

In [53]:
t2_survival_submission.head()

,acc_id,survival_time
0,43337,64
2,16892,64
6,13584,27
11,125768,49
13,104465,64


In [54]:
print(t1_DTC_pred.min())
print(t1_DTC_pred.max())
print(t2_DTC_pred.min())
print(t2_DTC_pred.max())

1
64
1
64
